In [2]:
import os
if not os.path.exists("yolov5"):
  print("cloning from git...") 
  !git clone https://github.com/ultralytics/yolov5
  %cd yolov5

%cd yolov5

/home/onur/Programming/Bachelor/CaDaBo/yolov5


In [7]:
%pwd

'/home/onur/Programming/Bachelor/CaDaBo/yolov5'

In [ ]:
%pip install -Ur requirements.txt
%pip install roboflow comet_ml pyarrow

In [ ]:
import glob

for file in glob.glob("../weights/CaDaBo-2/*nostretch*adam*"):
    print(file)
    !python export.py --weights {file} --img 960 --include onnx


In [ ]:
# convert .pt weights to onnx weights for pi

!python export.py --weights "../weights/CaDaBo-2/citroen-high-long.pt" --img 960 --data "CaDaBo-2-6/data.yaml" --include onnx
!python export.py --weights "../weights/CaDaBo-2/citroen-augs-high-long.pt" --img 960 --data "CaDaBo-2-9/data.yaml" --include onnx


In [ ]:
# download all version of cadabo-2
from roboflow import Roboflow
rf = Roboflow(api_key="CmkPncwo7RqoVbRtH71I")
project = rf.workspace("onurbachelor").project("cadabo-2")

for i in range(1, 10):
    dataset = project.version(i).download("yolov5")

In [ ]:
import torch

print(f"Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
!python "val.py" --weights "../weights/CaDaBo-2/random-640-low.pt" --img 640 --data "./CaDaBo-2-3/data.yaml" --task test --verbose

In [39]:
path="../weights/CaDaBo-2/random-640-low.pt"
filename = os.path.splitext(os.path.basename(path))[0]
print(filename)

random-640-low


In [6]:
path="../weights/CaDaBo-2/citroen-nostretch-augs-adam.onnx"
task = os.path.splitext(os.path.basename(path))[0]
data="CaDaBo-2-8/data.yaml"

!python "val.py" --weights {path} --name {task} --img 960 --data {data} --task test --verbose --exist-ok 2>&1 \
    | tee "../results/CaDaBo-2/{task}.txt"


val: data=CaDaBo-2-8/data.yaml, weights=['../weights/CaDaBo-2/citroen-nostretch-augs-adam.onnx'], batch_size=32, imgsz=960, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=citroen-nostretch-augs-adam, exist_ok=True, half=False, dnn=False
YOLOv5 🚀 v7.0-278-g050c72c Python-3.11.4 torch-2.1.2+cu121 CPU

Loading ../weights/CaDaBo-2/citroen-nostretch-augs-adam.onnx for ONNX Runtime inference...
Forcing --batch-size 1 square inference (1,3,960,960) for non-PyTorch models
test: Scanning /home/onur/Programming/Bachelor/CaDaBo/yolov5/CaDaBo-2-8/test/labels.cache... 53 images, 3 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 53/53 [00:43<00:00,  1.22it/s]
                   all         53        275      0.77

In [8]:
path="../weights/CaDaBo-2/citroen-adam.onnx"
task = os.path.splitext(os.path.basename(path))[0]
data="CaDaBo-2-6/data.yaml"
parent = os.path.dirname(data)
print(parent)

!python "detect.py" --weights {path} --name {task} --img 960 --source "{parent}/test/images/2023-12-07-08-51-30_png.rf.1b04efc53111c062f429a4d000d2398e.jpg" --save-txt --save-conf --exist-ok 2>&1

CaDaBo-2-6
detect: weights=['../weights/CaDaBo-2/citroen-adam.onnx'], source=CaDaBo-2-6/test/images/2023-12-07-08-51-30_png.rf.1b04efc53111c062f429a4d000d2398e.jpg, data=data/coco128.yaml, imgsz=[960, 960], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=citroen-adam, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-278-g050c72c Python-3.11.4 torch-2.1.2+cu121 CPU

Loading ../weights/CaDaBo-2/citroen-adam.onnx for ONNX Runtime inference...
image 1/1 /home/onur/Programming/Bachelor/CaDaBo/yolov5/CaDaBo-2-6/test/images/2023-12-07-08-51-30_png.rf.1b04efc53111c062f429a4d000d2398e.jpg: 960x960 1 airbag, 1 battery, 4 handbrakes, 1 mil, 1 oil, 1 seatbelt, 1 tirepressure, 844.7ms
Speed: 24.9ms pre-process, 844.7ms infere

In [ ]:
import glob
from PIL import Image

for image in glob.glob("runs/detect/exp4/*.jpg"):
  print(image)
  im= Image.open(image)
  display(im)